# Import library

In [1]:
import pandas as pd  # type: ignore # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # type: ignore # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
 
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks
 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia
 
from wordcloud import WordCloud  # type: ignore # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
 
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /home/yelf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/yelf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Scrapping Dataset

In [7]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort, reviews  # type: ignore
 
# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreviewAll = reviews_all(
    'com.byu.id',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=1000             # Jumlah maksimum ulasan yang ingin diambil
)

# ganti fungsi reviews_all ke reviews karena fungsi reviews_all akan meengambil semua data
scrapreview = reviews(
    'com.byu.id',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=199             # Jumlah maksimum ulasan yang ingin diambil
)

In [8]:
print("Tipe data menggunakan reviews_all : ", type(scrapreviewAll))
print("Tipe data menggunakan reviews     : ", type(scrapreview))

Tipe data menggunakan reviews_all :  <class 'list'>
Tipe data menggunakan reviews     :  <class 'tuple'>


In [13]:
# Menyimpan ulasan dalam file CSV
import csv
 
with open('dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapreview[0]:   # penambahan indexing karena tipe data tuple bukan list
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

# Loading Dataset

In [15]:
app_reviews_df = pd.DataFrame(scrapreview[0])
app_reviews_df.shape
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,2f4db43e-57d9-44cd-a861-472fa42e1fa3,Vian Oktavian,https://play-lh.googleusercontent.com/a/ACg8oc...,"Sinyal sering Ilang2 sendiri, ga stabil padaha...",1,62,1.55.0,2024-07-17 12:03:30,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-07-17 15:23:05,1.55.0
1,745064f3-5896-4b17-baf7-525ca5d6d8d2,Avrory,https://play-lh.googleusercontent.com/a-/ALV-U...,Selama 3 tahun pakai tidak ada perkembangan da...,2,52,1.54.1,2024-06-27 03:15:58,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-06-27 21:56:35,1.54.1
2,0acdcd22-28d4-4617-a607-e45743fd08be,Iqbal Kurniawan,https://play-lh.googleusercontent.com/a/ACg8oc...,waduuh kenapa sinyal byu jadi gni udh lebih dr...,1,88,1.54.1,2024-07-02 20:56:31,"Hai Kak, maaf nih terkait kendala jaringan, bi...",2024-07-02 21:16:39,1.54.1
3,19230435-a1ae-4db1-9e95-8b5b909f2726,Riki Ardian Je,https://play-lh.googleusercontent.com/a-/ALV-U...,"Saya pake by.u dari 2019, makin sini makin jel...",1,9,1.55.0,2024-07-14 04:17:37,"Hi Kak maaf udah bikin engga nyaman, saat ini ...",2024-07-14 14:39:32,1.55.0
4,ae75ea02-47b6-43a6-9853-344bb45f0455,dzulhajji maulana,https://play-lh.googleusercontent.com/a/ACg8oc...,"Perbaikin dulu aplikasinya, jaringan stabil be...",3,60,1.55.0,2024-06-26 23:25:12,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-06-27 21:56:30,1.55.0


In [16]:
# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview[0])

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape
print("Baris : {} & Kolom : {}".format(jumlah_ulasan, jumlah_kolom))

Baris : 199 & Kolom : 11


In [17]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              199 non-null    object        
 1   userName              199 non-null    object        
 2   userImage             199 non-null    object        
 3   content               199 non-null    object        
 4   score                 199 non-null    int64         
 5   thumbsUpCount         199 non-null    int64         
 6   reviewCreatedVersion  199 non-null    object        
 7   at                    199 non-null    datetime64[ns]
 8   replyContent          190 non-null    object        
 9   repliedAt             190 non-null    datetime64[ns]
 10  appVersion            199 non-null    object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 17.2+ KB


In [18]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

print("Baris : {} & Kolom : {}".format(jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat))

Baris : 190 & Kolom : 11


# Preprocessing Text
Fungsi-fungsi yang disediakan akan sangat berguna untuk langkah-langkah preprocessing teks. Mari kita jelaskan masing-masing fungsi dengan lebih singkat.

1. cleaningText(text): Membersihkan teks dengan menghapus mention, hashtag, RT (retweet), tautan (link), angka, dan tanda baca. Selain itu, karakter newline diganti dengan spasi dan spasi ekstra pada awal dan akhir teks dihapus.
2. casefoldingText(text): Mengonversi semua karakter dalam teks menjadi huruf kecil (lowercase) untuk membuat teks menjadi seragam.
3. tokenizingText(text): Memecah teks menjadi daftar kata atau token. Ini membantu dalam mengurai teks menjadi komponen-komponen dasar untuk analisis lebih lanjut.
4. filteringText(text): Menghapus kata-kata berhenti (stopwords) dalam teks. Daftar kata-kata berhenti telah diperbarui dengan beberapa kata tambahan.
5. stemmingText(text): Menerapkan stemming pada teks, yakni mengurangi kata-kata menjadi bentuk dasarnya. Anda menggunakan pustaka Sastrawi untuk melakukan stemming dalam bahasa Indonesia.
6. toSentence(list_words): Menggabungkan daftar kata-kata menjadi sebuah kalimat.

In [22]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer import StemmerFactory
 
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
 
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text
 
def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text
 
def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text
 
def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text
 
def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    # Memecah teks menjadi daftar kata
    words = text.split()
 
    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]
 
    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)
 
    return stemmed_text
 
def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [24]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [26]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)
 
# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
 
# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
 
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
 
# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)
 
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [27]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,2f4db43e-57d9-44cd-a861-472fa42e1fa3,Vian Oktavian,https://play-lh.googleusercontent.com/a/ACg8oc...,"Sinyal sering Ilang2 sendiri, ga stabil padaha...",1,62,1.55.0,2024-07-17 12:03:30,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-07-17 15:23:05,1.55.0,Sinyal sering Ilang sendiri ga stabil padahal ...,sinyal sering ilang sendiri ga stabil padahal ...,sinyal sering ilang sendiri ga stabil padahal ...,"[sinyal, sering, ilang, sendiri, ga, stabil, p...","[sinyal, ilang, stabil, kota, tolong, perbaiki...",sinyal ilang stabil kota tolong perbaiki u har...
1,745064f3-5896-4b17-baf7-525ca5d6d8d2,Avrory,https://play-lh.googleusercontent.com/a-/ALV-U...,Selama 3 tahun pakai tidak ada perkembangan da...,2,52,1.54.1,2024-06-27 03:15:58,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-06-27 21:56:35,1.54.1,Selama tahun pakai tidak ada perkembangan dal...,selama tahun pakai tidak ada perkembangan dal...,selama tahun pakai tidak ada perkembangan dala...,"[selama, tahun, pakai, tidak, ada, perkembanga...","[pakai, perkembangan, layanan, internet, kecep...",pakai perkembangan layanan internet kecepatan ...
2,0acdcd22-28d4-4617-a607-e45743fd08be,Iqbal Kurniawan,https://play-lh.googleusercontent.com/a/ACg8oc...,waduuh kenapa sinyal byu jadi gni udh lebih dr...,1,88,1.54.1,2024-07-02 20:56:31,"Hai Kak, maaf nih terkait kendala jaringan, bi...",2024-07-02 21:16:39,1.54.1,waduuh kenapa sinyal byu jadi gni udh lebih dr...,waduuh kenapa sinyal byu jadi gni udh lebih dr...,waduuh kenapa sinyal byu jadi gni udh lebih dr...,"[waduuh, kenapa, sinyal, byu, jadi, gni, udh, ...","[waduuh, sinyal, byu, gni, udh, dri, bln, paka...",waduuh sinyal byu gni udh dri bln pakai byu mi...
3,19230435-a1ae-4db1-9e95-8b5b909f2726,Riki Ardian Je,https://play-lh.googleusercontent.com/a-/ALV-U...,"Saya pake by.u dari 2019, makin sini makin jel...",1,9,1.55.0,2024-07-14 04:17:37,"Hi Kak maaf udah bikin engga nyaman, saat ini ...",2024-07-14 14:39:32,1.55.0,Saya pake byu dari makin sini makin jelek buk...,saya pake byu dari makin sini makin jelek buk...,saya pake byu dari makin sini makin jelek buka...,"[saya, pake, byu, dari, makin, sini, makin, je...","[pake, byu, jelek, berkembang, paket, murah, j...",pake byu jelek berkembang paket murah jaringan...
4,ae75ea02-47b6-43a6-9853-344bb45f0455,dzulhajji maulana,https://play-lh.googleusercontent.com/a/ACg8oc...,"Perbaikin dulu aplikasinya, jaringan stabil be...",3,60,1.55.0,2024-06-26 23:25:12,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-06-27 21:56:30,1.55.0,Perbaikin dulu aplikasinya jaringan stabil beg...,perbaikin dulu aplikasinya jaringan stabil beg...,perbaikin dulu aplikasinya jaringan stabil beg...,"[perbaikin, dulu, aplikasinya, jaringan, stabi...","[perbaikin, aplikasinya, jaringan, stabil, buk...",perbaikin aplikasinya jaringan stabil buka apl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,869e7d73-05f9-4b89-850b-a7dcc76e909b,Musakif Anadz,https://play-lh.googleusercontent.com/a/ACg8oc...,Butuh perbaikannya ini aplikasi...Mau log in s...,1,0,1.55.0,2024-06-22 11:30:57,"Hai Kak, terkait kendalanya, Nindy saranin log...",2024-06-22 11:49:45,1.55.0,Butuh perbaikannya ini aplikasiMau log in susa...,butuh perbaikannya ini aplikasimau log in susa...,butuh perbaikannya ini aplikasimau log in susa...,"[butuh, perbaikannya, ini, aplikasimau, log, i...","[butuh, perbaikannya, aplikasimau, log, susahk...",butuh perbaikannya aplikasimau log susahkartu ...
195,6cf77928-ddd2-49b3-a090-e14b4c02c8cf,Nina maulana05,https://play-lh.googleusercontent.com/a-/ALV-U...,Ya Allah by U kenapa kamu susah banget di buka...,2,0,1.55.0,2024-07-02 18:19:49,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-07-02 21:17:19,1.55.0,Ya Allah by U kenapa kamu susah banget di bu

# Labelling